In [1]:
!pip install confluent-kafka

In [2]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import psycopg2
import json

In [3]:
config = {
    'bootstrap.servers':'pkc-12576z.us-west2.gcp.confluent.cloud:9092',
    'security.protocol':'SASL_SSL',
    'sasl.mechanisms' :'PLAIN',
    'sasl.username':'G5T2ZXKQM66VBO7N',
#'sasl.password':'CkRZda/nUNw7af/8NXHKBe8AGMHunMAcJ+AQF+SBYh6h4LT4V0jaHHfPZHOq+dtA',
    'group.id':'lkc-y3pyxk', #cluster_id
    'auto.offset.reset':'earliest'

}

In [12]:
consumer = Consumer(**config)


In [5]:
# Subscribe to the 'users' topic
topic = 'topic_0'
consumer.subscribe([topic])#

In [6]:
postgres_conf = {
    'dbname': 'defaultdb',
    'user': 'avnadmin',
    # 'your-database password',
    'host': 'pg-480bc8-anitahkembs-9be0.g.aivencloud.com',
    'port': '17508'
}


In [22]:
conn = psycopg2.connect(**postgres_conf)
cursor = conn.cursor()

# create_table = """
# CREATE TABLE tbl_messages (
#     id SERIAL PRIMARY KEY,
#     Open Varchar(255),
#     High Varchar(255),
#     Low Varchar(255),
#     Close Varchar(255),
#     Volume Varchar(255)
# );
# # """
# cursor.execute(create_table)
# conn.commit()

In [15]:
try:
    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                print('%% %s [%d] reached end at offset %d\n' %
                      (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            # Print the message key and
            print('Received message: key=%s value=%s' % (msg.key().decode('utf-8'), msg.value().decode('utf-8')))
except KeyboardInterrupt:
    pass
finally:
    # Close down consumer to commit final offsets.
    consumer.close()

Received message: key=2024-07-30 19:55:00 value={"1. open": "190.7500", "2. high": "191.0000", "3. low": "190.7500", "4. close": "191.0000", "5. volume": "241"}
Received message: key=2024-07-30 19:50:00 value={"1. open": "191.0600", "2. high": "191.0600", "3. low": "190.9700", "4. close": "191.0000", "5. volume": "805"}
Received message: key=2024-07-30 19:20:00 value={"1. open": "190.8100", "2. high": "191.3700", "3. low": "190.5000", "4. close": "191.3700", "5. volume": "520"}
Received message: key=2024-07-30 19:00:00 value={"1. open": "191.0400", "2. high": "191.0400", "3. low": "191.0400", "4. close": "191.0400", "5. volume": "464639"}
Received message: key=2024-07-30 19:10:00 value={"1. open": "190.7800", "2. high": "190.7800", "3. low": "190.7800", "4. close": "190.7800", "5. volume": "1"}
Received message: key=2024-07-30 18:55:00 value={"1. open": "191.3000", "2. high": "191.3000", "3. low": "190.7600", "4. close": "190.7600", "5. volume": "22"}
Received message: key=2024-07-30 1

In [19]:
def insert_data(data):
  values = '''
  insert into tbl_messages (Open, High, Low, Close, Volume)
  values (%s, %s, %s, %s, %s)

  '''
  cursor.execute(values, (data['1. open'], data[ '2. high'], data['3. low'], data['4. close'], data['5. volume']))
  conn.commit()
  print("Data inserted successfully")



In [26]:
consumer = Consumer(**config)
topic = 'topic_0'
consumer.subscribe([topic])

In [27]:
conn = psycopg2.connect(**postgres_conf)
cursor = conn.cursor()
# Consume messages
try:
    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                print('%% %s [%d] reached end at offset %d\n' %
                      (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            # Print the message key and value
            print('Received message: key=%s value=%s' % (msg.key().decode('utf-8') if msg.key() else None, msg.value().decode('utf-8') if msg.value() else None))
            data = msg.value().decode('utf-8')
            msg = json.loads(data)

            # Insert data into PostgreSQL
            insert_data(msg)

except KeyboardInterrupt:
    pass
finally:
    # Close down consumer to commit final offsets.
    consumer.close()
    # Close PostgreSQL connection
    cursor.close()
    conn.close()

Received message: key=2024-07-30 15:15:00 value={"1. open": "190.6000", "2. high": "190.7800", "3. low": "190.6000", "4. close": "190.7200", "5. volume": "18070"}
Data inserted successfully
Received message: key=2024-07-30 15:05:00 value={"1. open": "190.4300", "2. high": "190.5200", "3. low": "190.3900", "4. close": "190.4700", "5. volume": "18442"}
Data inserted successfully
Received message: key=2024-07-30 14:45:00 value={"1. open": "190.5400", "2. high": "190.5400", "3. low": "190.2650", "4. close": "190.3000", "5. volume": "31844"}
Data inserted successfully
Received message: key=2024-07-30 14:20:00 value={"1. open": "190.8950", "2. high": "190.9300", "3. low": "190.7700", "4. close": "190.8300", "5. volume": "22304"}
Data inserted successfully
Received message: key=2024-07-30 14:15:00 value={"1. open": "190.6900", "2. high": "190.9300", "3. low": "190.6750", "4. close": "190.8950", "5. volume": "18405"}
Data inserted successfully
Received message: key=2024-07-30 14:00:00 value={"